In [42]:
import random
#import numpy as np
#import matplotlib.pyplot as plt
import pandas as pd
#import streamlit as st

# 1 per unique device
class Device:
    def __init__(self):
        self.device_model = data.nextDevice()
        self.lifetime = int(data.table.loc[data.table[data.table['Model Name'] == self.device_model].index, 'Life Time'])
        self.purchase_price = int(data.table.loc[data.table[data.table['Model Name'] == self.device_model].index, 'Purchase Price'])
        
        self.credit_payback_duration = 36
        self.total_credit_debt = self.purchase_price * 1.1   
        self.inventory_value = self.purchase_price
        self.monthly_credit_payback = self.total_credit_debt / 36

    def sell (self):
        finance.device_sales += self.purchase_price * 0.35
        print("sell command executed")
        data.devices_total -= 1
        data.devices_sold += 1

    def rentalPrice(self, term):
        self.rental_price = int(data.table.loc[data.table[data.table['Model Name'] == self.device_model].index, '{}'.format(term)])

    def reduceLifetime (self):
         self.lifetime = self.lifetime -1
    
    def depreciate (self):
        if self.lifetime == 0:
            self.inventory_value = 0
        else:
            self.inventory_value = self.inventory_value * (1-(1/self.lifetime))
        
# 1 per unique user
class User:
    def __init__(self):
        self.term = data.nextTerm()
        self.membership_type = self.term
        self.left_forever = False

    def reduceTerm (self):
        self.term = self.term -1

# 1 per simulation
class Data:
    def __init__(self):
        self.table = pd.read_csv('Siradaki_Price_List.csv',sep=';')
        self.users_total = 0
        self.devices_total = 0
        self.devices_sold = 0
        self.terms = random.choices([1,3,6,12], weights = [5,5,10,80], k = 100)
        self.devices = random.choices(self.table['Model Name'], weights = self.table['Rental Weight'], k = 100)
        self.id_dictionary = {}
        self.inventory = {}
        self.inventory_at_user = {}
        self.left_user = 0
    
    def nextDevice(self):
        return str(self.devices[self.devices_total % 100])

    def nextTerm(self):
        return int(self.terms[self.users_total % 100])
    
    def createMembership(self):
        user_id = "user_" + str(self.users_total)
        new_device_id = "device_" + str(self.devices_total)

        #inventory_length = len(self.inventory)

        if len(list(self.inventory)) >> 0: 
            devices_that_match = []

            for device_id in list(self.inventory):
                if self.inventory[device_id] == self.nextDevice() and globals()[device_id].lifetime > 0:
                    devices_that_match.append(device_id)
            
            if len(devices_that_match) >> 0:
                self.id_dictionary[user_id] = devices_that_match[0]
                del self.inventory[devices_that_match[0]]
                print("match! item deleted {} from inventory".format(devices_that_match[0]))
                self.inventory_at_user[devices_that_match[0]] = self.nextDevice()
                new_device_id = devices_that_match[0]


                #print('envanter envanterden verdi', self.inventory , "\n\n")
                #print('envanterden verdi', self.inventory_at_user , "\n\n")
                #print('id dict', self.id_dictionary, "\n\n")
            else:     
                self.createDevice()
                self.inventory_at_user[new_device_id] = self.nextDevice()
                self.id_dictionary[user_id] = new_device_id
                print("envanterde uyan urun yok ", user_id, " ", new_device_id)
                #print('envanter yok yeni yarattı', self.inventory , "\n\n")
                #print('yok yeni yarattı', self.inventory_at_user , "\n\n")
                #print('id dict', self.id_dictionary, "\n\n")
                        
        else:
            self.createDevice()
            self.inventory_at_user[new_device_id] = self.nextDevice()
            self.id_dictionary[user_id] = new_device_id
            print("inventory bos")
            #print('envanter yok', self.inventory_at_user , "\n\n")            
        
        self.createUser()
        #if random.random() < 0.5:
        #   self.createMembership()
        print("created membership with ", user_id, new_device_id)

    def createUser(self):
        user_id = "user_" + str(self.users_total)
        globals()[user_id] = User()
        self.users_total += 1

    def createDevice(self):
        device_id = "device_" + str(self.devices_total)
        globals()[device_id] = Device()
        self.devices_total += 1

    def devicesLifetime(self):
        for x in range(self.devices_total):
            device_id = 'device_' + str(x)
            globals()[device_id].reduceLifetime()
    
    def inventoryCheck(self):
        for device_id in list(self.inventory):
            print("inventory working" , device_id)
            print("with lifetime", globals()[device_id].lifetime)
            if globals()[device_id].lifetime <= 0:
                globals()[device_id].sell()
                print("a device got sold", device_id)
                del self.inventory[device_id]

    def userJobs(self):
        for user_id in list(self.id_dictionary):
            device_id = self.id_dictionary[user_id]
            user_instance = globals()[user_id]
            device_instance = globals()[device_id]

            finance.rent_income += float(self.table.loc[self.table[data.table['Model Name'] == device_instance.device_model].index, '{}'.format(user_instance.membership_type)])
                
            if device_instance.credit_payback_duration > 0:
                finance.credit_costs += device_instance.monthly_credit_payback
                device_instance.total_credit_debt -= device_instance.monthly_credit_payback
                device_instance.credit_payback_duration -= 1

            user_instance.reduceTerm()
            #device_instance.reduceLifetime()
            # user term bittiyse
            if user_instance.term <= 0 and user_instance.left_forever == False:
                if random.random() <= 0.5:
                    user_instance.term += user_instance.membership_type
                else:
                    user_instance.left_forever = True
                    print('giden user', user_id, device_id)
                    self.left_user += 1
                    self.inventory[device_id] = device_instance.device_model
                    #print('envanter - user left', self.inventory , "\n\n")
                    #print('id dict', self.id_dictionary)
                    #print(self.id_dictionary[user_id])
                    print(self.inventory_at_user[device_id])
                    del self.inventory_at_user[device_id]
                    del self.id_dictionary[user_id]

                # user term devam ediyorsa
                #else:
                    #finance.rent_income += device_instance.rentalPrice(user_instance.membership_type)        
        
# sigorta + bakim eklenecek
class Finance:
    def __init__(self):
        self.device_sales = 0
        self.rent_income = 0
        self.credit_costs = 0
        self.gross = self.device_sales + self.rent_income - self.credit_costs

finance = Finance()
data = Data()

for i in range(60):
    print("month {}\n".format(i))
    data.devicesLifetime()
    data.userJobs()
    data.inventoryCheck()
    for j in range(10):
        data.createMembership()




print('user sayısı', data.users_total)
print('id dict', len(data.id_dictionary))
print('envanter', len(data.inventory))
print('rented out', len(data.inventory_at_user))
print('sold', data.devices_sold)
print('gone users', data.left_user)
print('total devices',data.devices_total)
print(data.inventory)

if data.users_total == data.left_user + len(data.id_dictionary):
    print('test 1 passed')
if data.devices_total == data.devices_sold + len(data.inventory) + len(data.inventory_at_user):
    print('test 2 passed')
if len(data.inventory_at_user) == len(data.id_dictionary):
    print('test 3 passed')

ce_187
match! item deleted device_171 from inventory
created membership with  user_369 device_171
month 37

giden user user_130 device_111
Iphone 12 Pro Max 128 GB
giden user user_134 device_14
Iphone 11 Pro 64 GB
giden user user_139 device_116
Samsung Galaxy Z Fold 2 256 GB
giden user user_252 device_13
Playstation 5
giden user user_254 device_159
Apple Watch Series 6 GPS 44mm
giden user user_255 device_98
Apple Watch Series 6 GPS 44mm
giden user user_256 device_80
Apple Watch Series 6 GPS 44mm
giden user user_258 device_160
Playstation 5
giden user user_259 device_161
Iphone 12 Pro Max 128 GB
inventory working device_39
with lifetime 3
inventory working device_139
with lifetime 18
inventory working device_119
with lifetime 13
inventory working device_41
with lifetime 3
inventory working device_142
with lifetime 18
inventory working device_149
with lifetime 8
inventory working device_152
with lifetime 8
inventory working device_164
with lifetime 26
inventory working device_127
with li

KeyError: 'device_178'

In [48]:
x = {}
x[0] = 4
x[1] = 5
for i in x:
    print(i)

0
1


In [10]:
random.random()

0.5263239444743002

In [12]:
class X:
    def yo(self):
        print("y")
    def oy(self):
        self.yo()

x = X()
x.oy()

y
